In [5]:
from PIL import Image

def message_to_bin(message):
    binary_message = ''.join(format(ord(i), '08b') for i in message)
    return binary_message

def hide_message(image_path, message, output_path):
    image = Image.open(image_path)
    binary_message = message_to_bin(message)
    binary_message += '1111111111111110'  # Añade un delimitador al final del mensaje
    data = list(image.getdata())
    new_data = []
    message_index = 0

    for pixel in data:
        new_pixel = []
        for color in pixel[:3]:  # Ignoramos el canal alfa si existe
            if message_index < len(binary_message):
                new_color = color & 0b11111110  # Limpia el bit menos significativo
                new_color |= int(binary_message[message_index])  # Añade el bit del mensaje
                new_pixel.append(new_color)
                message_index += 1
            else:
                new_pixel.append(color)
        if len(pixel) == 4:  # Si hay un canal alfa, añádelo sin cambios
            new_pixel.append(pixel[3])
        new_data.append(tuple(new_pixel))

    new_image = Image.new(image.mode, image.size)
    new_image.putdata(new_data)
    new_image.save(output_path)

# Uso del programa
hide_message("../aa.png", "Hola Javier Pi", "../aa_con_mensaje.png")

In [6]:
def bin_to_message(binary_message):
    chars = [chr(int(binary_message[i:i+8], 2)) for i in range(0, len(binary_message), 8)]
    return ''.join(chars)

def extract_message(image_path):
    image = Image.open(image_path)
    data = list(image.getdata())
    binary_message = ""

    for pixel in data:
        for color in pixel[:3]:
            binary_message += str(color & 1)  # Toma el bit menos significativo

    # Busca el delimitador al final del mensaje
    delimiter = '1111111111111110'
    message_end = binary_message.find(delimiter)
    if message_end != -1:
        binary_message = binary_message[:message_end]
    
    return bin_to_message(binary_message)

# Uso del programa
print(extract_message("../aa_con_mensaje.png"))


Hola Javier Pi
